This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.4_tutorials/wallaroo-model-operations-tutorials/deploy/by-use-case/sentiment-analysis-hugging-face).

## Sentiment Analysis with Hugging Face Toxic Bert Model

The following tutorial demonstrates performing sentiment analysis via the Hugging Face model Toxic Bert.  This model inputs a set of text, and outputs a set of scores based on whether the text is considered toxic, hateful, etc.

This tutorial demonstrates:

* Uploading the model to a Wallaroo environment.
* Deploying the model via a Wallaroo pipeline.
* Performing a sample inference on the deployed model.
* Undeploy the model and return the resources back to the environment.

For access to these sample models and for a demonstration of how to use Wallaroo:

* Contact your Wallaroo Support Representative **OR**
* [Schedule Your Wallaroo.AI Demo Today](https://wallaroo.ai/request-a-demo/)

## Tutorial Steps

### Import libraries

The first step is to import the libraries required.

In [104]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework
from wallaroo.deployment_config import DeploymentConfigBuilder

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

import pyarrow as pa
import json
import datetime
import time
import zipfile

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [105]:
wl = wallaroo.Client(request_timeout=120)

### Set Workspace and Variables

The following creates or connects to an existing workspace, and sets it as the current workspace.  For more details on Wallaroo workspaces, see [Wallaroo Workspace Management Guide](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-optimize/wallaroo-workspace-management/).

We will set the variables used for our deployed LLM model, and the models used for our LLM Listener.

In [106]:
workspace_name = "sentiment-analysis-tutorial"  

wl.set_current_workspace(wl.get_workspace(workspace_name, create_if_not_exist=True))

{'name': 'sentiment-analysis-tutorial', 'id': 16, 'archived': False, 'created_by': 'john.hansarick@wallaroo.ai', 'created_at': '2025-05-01T19:38:22.227711+00:00', 'models': [], 'pipelines': [{'name': 'sentiment-analysis', 'create_time': datetime.datetime(2025, 5, 1, 19, 45, 26, 228651, tzinfo=tzutc()), 'definition': '[]'}]}

### Upload LLM Listener Models and Create a Monitoring Pipeline

This monitoring pipeline consists of a [Hugging Face sentiment analyzer](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/#wallaroo-supported-models) step.

The following model is used:

* `toxic_bert`: A [Hugging Face Text Classification](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/#wallaroo-supported-models) model that evaluates LLM outputs and outputs an array of scores including:
  * `identity_hate`
  * `insult`
  * `obscene`
  * `severe_toxic`
  * `threat`
  * `toxic`

We upload the model via the [`wallaroo.client.Client.upload_models`](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/) method.

In [107]:
pipeline.undeploy()

name,sentiment-analysis
created,2025-05-01 19:45:26.228651+00:00
last_updated,2025-05-01 20:57:26.958841+00:00
deployed,False
workspace_id,16
workspace_name,sentiment-analysis-tutorial
arch,x86
accel,none
tags,
versions,"fbf54347-73f4-4042-a4ed-3371914c2e92, f5b2fae4-ffc8-4856-b942-c9f73a048ee6, 05736a38-9942-47e6-941c-177bbf712f84, 9063936f-ee24-4eba-a1c4-3618b905a031, 9924255d-9c92-4f92-9c70-441fd9675075, 5a3938e0-4eba-46b9-9102-c09ae063b29a, 58be18bc-6c7c-44b9-bf1e-3735b40f040b, 0342b954-41bf-4342-a5de-5740a5505c40, 1c94dad4-e77d-4f1a-81e6-5c21870fde08, b061ff23-ba26-4682-a738-2a09b55c1cb8, 55a57661-6199-4175-9d87-a02b4603a270, ea31b3aa-c51a-4604-bed2-7d68ad75dc56, 7b560178-00ce-496d-953b-3cbfcebaed4b, a928dffe-85f9-4f1c-8270-59ac6edfa5a0, 1f435615-77d2-49a0-8775-04d169a85bcd, 82fba0f8-4910-4106-88b9-2ef6235aca16, 0a633695-c33e-46d6-be1f-757690628702, 0b30d696-5c54-43ad-b099-85a7454e231b, 7303a912-5f05-4297-bbeb-09734fe730c9, 00df5dd7-d732-4537-b883-7dae579b4e95, 818d5c76-5aeb-422a-ba9f-6d43ccb337c0, c4555c1c-2c42-4067-8a25-1c3db4e5a682"
steps,


In [108]:
# upload the sentiment analyzer

input_schema = pa.schema([
     pa.field('inputs', pa.string()), # required
     pa.field('top_k', pa.int64()),  
])

output_schema = pa.schema([
     pa.field('label', pa.list_(pa.string(), list_size=6)), # list with a number of items same as top_k, list_size can be skipped but may lead in worse performance
     pa.field('score', pa.list_(pa.float64(), list_size=6)), # list with a number of items same as top_k, list_size can be skipped but may lead in worse performance
])

framework=Framework.HUGGING_FACE_TEXT_CLASSIFICATION
model_name = "toxic-bert-analysis"
model_file_name = './models/unitary-toxic-bert.zip'

bert_model = wl.upload_model(model_name,
                         model_file_name,
                         framework=framework,
                         input_schema=input_schema,
                         output_schema=output_schema,
                         convert_wait=False)


Once uploaded, the following loop verifies the conversion process is complete before proceeding.

In [109]:
while bert_model.status() != "ready" and bert_model.status() != "error":
    print(bert_model.status())
    time.sleep(10)
print(bert_model.status())

pending_load_container
attempting_load_container
attempting_load_container
attempting_load_container
attempting_load_container
attempting_load_container
attempting_load_container
attempting_load_container
attempting_load_container
attempting_load_container
attempting_load_container
ready


### Deploy the Models

We deploy with a [deployment configuration](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-deploy-model/wallaroo-model-operations-deploy-model-deployment-configuration/) and set the Hugging Face sentiment analyzer to 4 cpus and 8 Gi RAM.

We create the pipeline with the `build_pipeline` method, and add the model as the [pipeline steps](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-deploy-model/).

In [110]:
# this is the summarizer config 
deployment_config = wallaroo.DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .sidekick_cpus(bert_model, 4) \
    .sidekick_memory(bert_model, "8Gi") \
    .build()

pipeline_name = 'sentiment-analysis'
pipeline=wl.build_pipeline(pipeline_name)
pipeline.clear()
pipeline.add_model_step(bert_model)

name,sentiment-analysis
created,2025-05-01 19:45:26.228651+00:00
last_updated,2025-05-01 21:11:08.381157+00:00
deployed,False
workspace_id,16
workspace_name,sentiment-analysis-tutorial
arch,x86
accel,none
tags,
versions,"9dd0df40-f0c9-4dd1-a7ea-85521eca15f2, fbf54347-73f4-4042-a4ed-3371914c2e92, f5b2fae4-ffc8-4856-b942-c9f73a048ee6, 05736a38-9942-47e6-941c-177bbf712f84, 9063936f-ee24-4eba-a1c4-3618b905a031, 9924255d-9c92-4f92-9c70-441fd9675075, 5a3938e0-4eba-46b9-9102-c09ae063b29a, 58be18bc-6c7c-44b9-bf1e-3735b40f040b, 0342b954-41bf-4342-a5de-5740a5505c40, 1c94dad4-e77d-4f1a-81e6-5c21870fde08, b061ff23-ba26-4682-a738-2a09b55c1cb8, 55a57661-6199-4175-9d87-a02b4603a270, ea31b3aa-c51a-4604-bed2-7d68ad75dc56, 7b560178-00ce-496d-953b-3cbfcebaed4b, a928dffe-85f9-4f1c-8270-59ac6edfa5a0, 1f435615-77d2-49a0-8775-04d169a85bcd, 82fba0f8-4910-4106-88b9-2ef6235aca16, 0a633695-c33e-46d6-be1f-757690628702, 0b30d696-5c54-43ad-b099-85a7454e231b, 7303a912-5f05-4297-bbeb-09734fe730c9, 00df5dd7-d732-4537-b883-7dae579b4e95, 818d5c76-5aeb-422a-ba9f-6d43ccb337c0, c4555c1c-2c42-4067-8a25-1c3db4e5a682"
steps,


With the pipeline set, we deploy the pipeline with the defined deployment configuration.  This allocates the resources from the cluster for the LLM Listener models use.

Once the models are deployed, we check the status and verify it's running.

In [111]:
pipeline.deploy(deployment_config=deployment_config, wait_for_status=False)

Deployment initiated for sentiment-analysis. Please check pipeline status.


name,sentiment-analysis
created,2025-05-01 19:45:26.228651+00:00
last_updated,2025-05-01 21:11:15.044982+00:00
deployed,True
workspace_id,16
workspace_name,sentiment-analysis-tutorial
arch,x86
accel,none
tags,
versions,"ffe6b4f9-99c3-4f35-b72e-8a16cb211c5b, 9dd0df40-f0c9-4dd1-a7ea-85521eca15f2, fbf54347-73f4-4042-a4ed-3371914c2e92, f5b2fae4-ffc8-4856-b942-c9f73a048ee6, 05736a38-9942-47e6-941c-177bbf712f84, 9063936f-ee24-4eba-a1c4-3618b905a031, 9924255d-9c92-4f92-9c70-441fd9675075, 5a3938e0-4eba-46b9-9102-c09ae063b29a, 58be18bc-6c7c-44b9-bf1e-3735b40f040b, 0342b954-41bf-4342-a5de-5740a5505c40, 1c94dad4-e77d-4f1a-81e6-5c21870fde08, b061ff23-ba26-4682-a738-2a09b55c1cb8, 55a57661-6199-4175-9d87-a02b4603a270, ea31b3aa-c51a-4604-bed2-7d68ad75dc56, 7b560178-00ce-496d-953b-3cbfcebaed4b, a928dffe-85f9-4f1c-8270-59ac6edfa5a0, 1f435615-77d2-49a0-8775-04d169a85bcd, 82fba0f8-4910-4106-88b9-2ef6235aca16, 0a633695-c33e-46d6-be1f-757690628702, 0b30d696-5c54-43ad-b099-85a7454e231b, 7303a912-5f05-4297-bbeb-09734fe730c9, 00df5dd7-d732-4537-b883-7dae579b4e95, 818d5c76-5aeb-422a-ba9f-6d43ccb337c0, c4555c1c-2c42-4067-8a25-1c3db4e5a682"
steps,toxic-bert-analysis


In [ ]:
time.sleep(30)

while pipeline.status()['status'] != 'Running':
    time.sleep(30)
    print("Waiting for deployment.")
    display(pipeline.status()['status'])
display(pipeline.status()['status'])

Waiting for deployment.


'Starting'

Waiting for deployment.


'Starting'

Waiting for deployment.


...

Waiting for deployment.


'Starting'

Waiting for deployment.


'Running'

'Running'

### Sample Inference

With sentiment analysis models deployed, we perform an inference via a sample pandas DataFrame via the [pipeline inference from file method](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-serve/).

These results are returned as a pandas DataFrame, with the outputs shown in the `out.*` columns.

In [113]:
results = pipeline.infer_from_file("./data/sample_input.json")
results

,time,in.inputs,in.top_k,out.label,out.score,anomaly.count
0,2025-05-01 21:15:23.449,"Wallaroo.AI is an AI platform that enables developers to build, deploy, and manage AI and machine learning models at scale. It provides a cloud-based infrastructure for building, training, and deploying AI models, as well as a set of tools and APIs for integrating AI into various applications.\n\nWallaroo.AI is designed to make it easy for developers to build and deploy AI models, regardless of their level of expertise in machine learning. It provides a range of features, including support for popular machine learning frameworks such as TensorFlow and PyTorch, as well as a set of pre-built AI models and APIs for common use cases such as image and speech recognition, natural language processing, and predictive analytics.\n\nWallaroo.AI is particularly well-suited for developers who are looking to build AI-powered applications, but may not have extensive expertise in machine learning or AI development. It provides a range of tools and resources to help developers get started with building AI-powered applications, including a cloud-based development environment, a set of pre-built AI models and APIs, and a range of tutorials and documentation.",6,"[toxic, obscene, insult, identity_hate, threat, severe_toxic]","[0.0006922021857462823, 0.00018145183275919408, 0.00017831838340498507, 0.00014974642544984818, 0.00013229971227701753, 0.00012232053268235177]",0


The following allows us to view just the output results using the `models.postprocess` Python script, showing how the sentiment analysis model scored our input.

In [114]:
import models.postprocess
import importlib
importlib.reload(models.postprocess)

df = pd.DataFrame(models.postprocess.process_data(results))
df

,toxic,obscene,insult,identity_hate,threat,severe_toxic
0,[0.0006922021857462823],[0.00018145183275919408],[0.00017831838340498507],[0.00014974642544984818],[0.00013229971227701753],[0.00012232053268235177]


### Undeploy

With the tutorial complete, we undeploy the models to return the resources back to the cluster.

In [115]:
pipeline.undeploy()

name,sentiment-analysis
created,2025-05-01 19:45:26.228651+00:00
last_updated,2025-05-01 21:11:15.044982+00:00
deployed,False
workspace_id,16
workspace_name,sentiment-analysis-tutorial
arch,x86
accel,none
tags,
versions,"ffe6b4f9-99c3-4f35-b72e-8a16cb211c5b, 9dd0df40-f0c9-4dd1-a7ea-85521eca15f2, fbf54347-73f4-4042-a4ed-3371914c2e92, f5b2fae4-ffc8-4856-b942-c9f73a048ee6, 05736a38-9942-47e6-941c-177bbf712f84, 9063936f-ee24-4eba-a1c4-3618b905a031, 9924255d-9c92-4f92-9c70-441fd9675075, 5a3938e0-4eba-46b9-9102-c09ae063b29a, 58be18bc-6c7c-44b9-bf1e-3735b40f040b, 0342b954-41bf-4342-a5de-5740a5505c40, 1c94dad4-e77d-4f1a-81e6-5c21870fde08, b061ff23-ba26-4682-a738-2a09b55c1cb8, 55a57661-6199-4175-9d87-a02b4603a270, ea31b3aa-c51a-4604-bed2-7d68ad75dc56, 7b560178-00ce-496d-953b-3cbfcebaed4b, a928dffe-85f9-4f1c-8270-59ac6edfa5a0, 1f435615-77d2-49a0-8775-04d169a85bcd, 82fba0f8-4910-4106-88b9-2ef6235aca16, 0a633695-c33e-46d6-be1f-757690628702, 0b30d696-5c54-43ad-b099-85a7454e231b, 7303a912-5f05-4297-bbeb-09734fe730c9, 00df5dd7-d732-4537-b883-7dae579b4e95, 818d5c76-5aeb-422a-ba9f-6d43ccb337c0, c4555c1c-2c42-4067-8a25-1c3db4e5a682"
steps,toxic-bert-analysis


For access to these sample models and for a demonstration:

* Contact your Wallaroo Support Representative **OR**
* [Schedule Your Wallaroo.AI Demo Today](https://wallaroo.ai/request-a-demo/)